<a href="https://colab.research.google.com/github/OneFineStarstuff/Cosmic-Brilliance/blob/main/simulate_and_plot_extended_py.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install numpy pandas matplotlib

In [ ]:
#!/usr/bin/env python3
"""
simulate_and_plot_extended.py

Runs a simulation of log_output and log_beyond_field, saves data,
and creates multiple plots for analysis and comparison.
"""

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

def simulate(
    total_steps: int,
    init_log_output: float,
    init_log_beyond: float,
    growth_per_step: float,
    breach_step: int,
    breach_value: float
) -> pd.DataFrame:
    """
    Generate time series for log_output and log_beyond_field.
    """
    steps = np.arange(1, total_steps + 1)
    log_out = init_log_output + growth_per_step * (steps - 1)
    log_bey = np.full(total_steps, init_log_beyond, dtype=float)

    # Apply breach jump
    idx = breach_step - 1
    if 0 <= idx < total_steps:
        log_bey[idx:] = breach_value

    return pd.DataFrame({
        'step': steps,
        'log_output': log_out,
        'log_beyond_field': log_bey
    })

def save_to_csv(df: pd.DataFrame, filename: str):
    """Save simulation DataFrame to CSV."""
    df.to_csv(filename, index=False)
    print(f"Data saved to {filename}")

def downsample(df: pd.DataFrame, stride: int = 100) -> pd.DataFrame:
    """Return every `stride`-th row for faster plotting."""
    return df.iloc[::stride].reset_index(drop=True)

def plot_logs(df: pd.DataFrame, filename: str, title: str = 'Log vs Step'):
    """Plot log_output & log_beyond_field."""
    plt.figure(figsize=(10, 6))
    plt.plot(df['step'], df['log_output'], label='log_output', color='tab:blue', alpha=0.7)
    plt.plot(df['step'], df['log_beyond_field'], label='log_beyond_field', color='tab:orange', alpha=0.7)
    plt.xlabel('Step')
    plt.ylabel('Log Value')
    plt.title(title)
    plt.legend(loc='best')
    plt.tight_layout()
    plt.savefig(filename, dpi=200)
    plt.close()
    print(f"Plot saved to {filename}")

def plot_with_annotation(df: pd.DataFrame, breach_step: int, filename: str):
    """Plot logs and annotate the breach point."""
    plt.figure(figsize=(10, 6))
    plt.plot(df['step'], df['log_output'], label='log_output', color='tab:blue', alpha=0.7)
    plt.plot(df['step'], df['log_beyond_field'], label='log_beyond_field', color='tab:orange', alpha=0.7)

    yb = df.loc[df.step == breach_step, 'log_beyond_field'].iloc[0]
    plt.scatter([breach_step], [yb], color='red', zorder=5)
    plt.annotate(f'Breach @ {breach_step}',
                 xy=(breach_step, yb),
                 xytext=(breach_step * 0.6, yb + 1),
                 arrowprops=dict(arrowstyle='->', color='red'))

    plt.xlabel('Step')
    plt.ylabel('Log Value')
    plt.title('Simulation with Breach Annotation')
    plt.legend(loc='upper left')
    plt.tight_layout()
    plt.savefig(filename, dpi=200)
    plt.close()
    print(f"Annotated plot saved to {filename}")

def plot_real_scale(df: pd.DataFrame, filename: str):
    """Convert log_beyond_field back to real scale and plot."""
    df_real = df.copy()
    df_real['beyond_field'] = np.exp(df_real['log_beyond_field'])

    plt.figure(figsize=(10, 6))
    plt.plot(df_real['step'], df_real['beyond_field'], color='tab:green')
    plt.yscale('log')
    plt.xlabel('Step')
    plt.ylabel('Beyond Field (exp)')
    plt.title('True “Beyond Field” vs. Steps')
    plt.tight_layout()
    plt.savefig(filename, dpi=200)
    plt.close()
    print(f"Real-scale plot saved to {filename}")

def overlay_runs(runs: dict, filename: str):
    """
    Overlay multiple simulations on one plot.
    `runs` is a dict: {label: DataFrame}
    """
    plt.figure(figsize=(10, 6))
    for label, df_run in runs.items():
        plt.plot(df_run['step'], df_run['log_beyond_field'], label=label, alpha=0.8)
    plt.xlabel('Step')
    plt.ylabel('log_beyond_field')
    plt.title('Comparison of Breach Levels')
    plt.legend()
    plt.tight_layout()
    plt.savefig(filename, dpi=200)
    plt.close()
    print(f"Comparison plot saved to {filename}")

def main():
    # Core parameters
    TOTAL_STEPS     = 1_000_000
    INIT_LOG_OUT    = 230.2585
    INIT_LOG_BEY    = 0.0
    GROWTH_PER_STEP = 0.00005
    BREACH_STEP     = 500_000

    # Run base simulation
    df_base = simulate(
        total_steps      = TOTAL_STEPS,
        init_log_output  = INIT_LOG_OUT,
        init_log_beyond  = INIT_LOG_BEY,
        growth_per_step  = GROWTH_PER_STEP,
        breach_step      = BREACH_STEP,
        breach_value     = 0.0001
    )

    # Persist data
    save_to_csv(df_base, 'logs.csv')

    # 1) Basic log plot
    plot_logs(df_base, 'logs_plot.png', title='log_output & log_beyond_field vs Steps')

    # 2) Annotated breach point
    plot_with_annotation(df_base, BREACH_STEP, 'annotated_plot.png')

    # 3) True-scale beyond_field
    plot_real_scale(df_base, 'beyond_field_plot.png')

    # 4) Downsampled plot
    df_ds = downsample(df_base, stride=100)
    plot_logs(df_ds, 'logs_downsampled.png', title='Downsampled log vs Step')

    # 5) Overlay two breach scenarios
    df_high = simulate(
        total_steps      = TOTAL_STEPS,
        init_log_output  = INIT_LOG_OUT,
        init_log_beyond  = INIT_LOG_BEY,
        growth_per_step  = GROWTH_PER_STEP,
        breach_step      = BREACH_STEP,
        breach_value     = 0.0010
    )
    overlay_runs({
        'breach=1e-4': df_base,
        'breach=1e-3': df_high
    }, 'compare_breaches.png')

if __name__ == '__main__':
    main()